In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl,np,pd,sklearn,tf,keras:
    print(module.__name__,module.__version__)

tf.test.is_gpu_available()

/Users/zhenwuzhou/environment/tf1_py3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/zhenwuzhou/environment/tf1_py3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/zhenwuzhou/environment/tf1_py3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/zhenwuzhou/environment/tf1_py3

1.13.1
sys.version_info(major=3, minor=7, micro=4, releaselevel='final', serial=0)
matplotlib 3.1.2
numpy 1.18.1
pandas 0.25.3
sklearn 0.22.1
tensorflow 1.13.1
tensorflow._api.v1.keras 2.2.4-tf


False

In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train_all,y_train_all),(x_test, y_test) = fashion_mnist.load_data()
x_valid,x_train = x_train_all[:5000],x_train_all[5000:]
y_valid,y_train = y_train_all[:5000],y_train_all[5000:]

print(x_train.shape,y_train.shape)
print(x_valid.shape,y_valid.shape)
print(x_test.shape,y_test.shape)
print(np.max(x_train),np.min(x_train))

# 这里对label数据类型进行更改，
y_train = np.asarray(y_train,dtype = np.int64)
y_valid = np.asarray(y_valid,dtype = np.int64)
y_test = np.asarray(y_test,dtype = np.int64)

(55000, 28, 28) (55000,)
(5000, 28, 28) (5000,)
(10000, 28, 28) (10000,)
255 0


In [3]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# x_train:[None,28,28] -> [None,784]
x_train_scaled = scaler.fit_transform(
    x_train.astype(np.float32).reshape(-1,1)).reshape(-1,28*28)
x_valid_scaled = scaler.fit_transform(
    x_valid.astype(np.float32).reshape(-1,1)).reshape(-1,28*28)
x_test_scaled = scaler.fit_transform(
    x_test.astype(np.float32).reshape(-1,1)).reshape(-1,28*28)
print(np.max(x_train_scaled),np.min(x_train_scaled))

2.0231433 -0.8105136


In [4]:
def make_dataset(images,labels,epochs,batch_size,shuffle = True):
    dataset = tf.data.Dataset.from_tensor_slices((images,labels))
    if shuffle:
        dataset.shuffle(10000)
    dataset = dataset.repeat(epochs).batch(batch_size)
    return dataset

In [8]:
# 那么要怎样才能在1.0中访问dataset呢
epochs = 10
batch_size = 20

# make_initializable_iterator可以重新初始化
# 所以这里用两个placeholder来树初始化dataset
images_placeholder = tf.placeholder(tf.float32, [None,28 *28])
labels_placeholder = tf.placeholder(tf.int64,[None])

dataset = make_dataset(images_placeholder,labels_placeholder,
                       epochs = epochs,
                       batch_size = batch_size)

# 创建initializable_iterator
dataset_iter = dataset.make_initializable_iterator()
x,y = dataset_iter.get_next() # 取出tensor
# 创建session来访问
with tf.Session() as sess:
    # 先去动态设置训练数据
    sess.run(dataset_iter.initializer, # 运行时传入dataset_iter.initializer
             feed_dict = { # 和需要动态设置给placeholder的数据
                 images_placeholder: x_train_scaled,
                 labels_placeholder: y_train
             })
    x_val,y_val = sess.run([x,y])
    print(x_val.shape)
    print(y_val.shape)
    
    # 在动态设置验证数据
    sess.run(dataset_iter.initializer,
             feed_dict = {
                 images_placeholder: x_valid_scaled,
                 labels_placeholder: y_valid
             })
    x_val,y_val = sess.run([x,y])
    print(x_val.shape)
    print(y_val.shape)

(20, 784)
(20,)
(20, 784)
(20,)
